### Dependencies to Install / Initial Checks





In [ ]:
!pip3 install datasets
!pip install numpy
!pip install sentence_transformers
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers sentence-transformers
import torch
from datasets import Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers, ParallelMode
from sentence_transformers.evaluation import TripletEvaluator
! pip install -U accelerate
! pip install -U transformers
import accelerate
import os

In [ ]:
!nvidia-smi

Tue Jun 25 04:07:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Make Sure GPU Instance is Working
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    try:
        x = torch.tensor([1.0, 2.0, 3.0], device='cuda')
        print(f"Tensor on CUDA: {x}")
        torch.cuda.empty_cache()
        print("torch.cuda.empty_cache() executed successfully.")
    except Exception as e:
        print(f"Error during CUDA operations: {e}")

CUDA available: True
Tensor on CUDA: tensor([1., 2., 3.], device='cuda:0')
torch.cuda.empty_cache() executed successfully.


### Setting Up Dataset

In [ ]:
#if missing place this txt file from the gdrive folder into the main area or change path if storing at different location.
file = open("/content/wikidateaware_v3_mini.txt", "r")

#dataset dictionary
dataset = {}
dataset["anchors"] = []
dataset["positive"] = []

#read dataset into dictionary
for line in file.readlines():
    if line == "Query|Document":
      continue
    else:
      if "|" in line:
        try:
          vals = line.split("|")
          query = vals[0]
          doc = vals[1]
          dataset["anchors"].append("search query: " + query)
          dataset["positive"].append("search document: " + doc)
        except:
          print("Error for line: ", line, line.count("|"))
dataset = Dataset.from_dict({
    "anchor": dataset["anchors"],
    "positive": dataset["positive"],
})
print(dataset)

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 1513
})


### Training

In [ ]:
#loading nomic-ai/nomic-embed-text-v1 embeddings model for finetuning
!pip install einops
model = SentenceTransformer(
    "nomic-ai/nomic-embed-text-v1",
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
loss = MultipleNegativesRankingLoss(model)

In [ ]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
# torch.distributed.init_process_group(rank = torch.cuda.device_count(), world_size=1)
print(accelerate.__version__)
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="/content/models/nomic_dateaware",
    # Optional training parameters:
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    warmup_steps=400,
    logging_first_step=True,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=10,
    logging_steps=1,
    run_name="nomic_dateaware",  # Will be used in W&B if `wandb` is installed
)
print(args.parallel_mode)
print("Initializing trainer...")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Both warmup_ratio and warmup_steps given, warmup_steps will override any effect of warmup_ratio during training


0.31.0
ParallelMode.NOT_PARALLEL
Initializing trainer...


In [ ]:
import transformers
transformers.logging.set_verbosity_debug()
transformers.utils.logging.enable_default_handler()
def main():
  # Set environment variables for debugging
  os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
  os.environ['TORCH_USE_CUDA_DSA'] = "1"

  # Print CUDA and PyTorch version info
  # print(f"PyTorch version: {torch.__version__}")
  # print(f"CUDA version: {torch.version.cuda}")
  # print(f"CUDA available: {torch.cuda.is_available()}")

  # # Check GPU memory status
  # print(torch.cuda.memory_summary())

  # # Print model, args, dataset, and loss for inspection
  # print("Model:", model)
  # print("Args:", args)
  # print("Dataset:", dataset)
  # print("Loss:", loss)

  # Proceed with trainer initialization
  try:
        #Restart Session using the commented lines if this cell fails.
      # import os
      # os._exit(00)
      trainer = SentenceTransformerTrainer(
          model=model,
          args=args,
          train_dataset=dataset,
          loss=loss,
          eval_dataset=dataset,
          evaluator=None,
      )
      trainer.add_callback(transformers.ProgressCallback)
      print("Trainer initialized successfully.")
      # Proceed with training
      trainer.train()
      model.save_pretrained("models/nomic_dateaware/final")
      print("Model saved successfully.")
  except RuntimeError as e:
      print("Caught RuntimeError:", e)
      print("Potential issue with CUDA kernel execution.")
if __name__ == "__main__":
    main()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using auto half precision backend
Currently training with a batch size of: 8
***** Running training *****
  Num examples = 1,513
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 380
  Number of trainable parameters = 136,731,648


Trainer initialized successfully.


  0%|          | 0/380 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss
100,0.000300,0.200723
200,0.009400,0.076758
300,1.176400,0.067655


{'loss': 0.0671, 'grad_norm': 20.38685417175293, 'learning_rate': 5.0000000000000004e-08, 'epoch': 0.01}
{'loss': 0.0, 'grad_norm': 0.003753146156668663, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.01}
{'loss': 0.0059, 'grad_norm': 1.0934797525405884, 'learning_rate': 1.5000000000000002e-07, 'epoch': 0.02}
{'loss': 0.0, 'grad_norm': 0.004538164008408785, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.02}
{'loss': 0.0051, 'grad_norm': 0.5697325468063354, 'learning_rate': 2.5000000000000004e-07, 'epoch': 0.03}
{'loss': 0.0086, 'grad_norm': 1.1038992404937744, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.03}
{'loss': 0.0005, 'grad_norm': 0.06338191032409668, 'learning_rate': 3.5000000000000004e-07, 'epoch': 0.04}
{'loss': 0.0077, 'grad_norm': 1.253654956817627, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.04}
{'loss': 0.0005, 'grad_norm': 0.0723206028342247, 'learning_rate': 4.5000000000000003e-07, 'epoch': 0.05}
{'loss': 0.0288, 'grad_norm': 6.5558905601501465,

...

{'loss': 0.0109, 'grad_norm': 2.507520914077759, 'learning_rate': 1.88e-05, 'epoch': 1.98}
{'loss': 0.0004, 'grad_norm': 0.0845104455947876, 'learning_rate': 1.885e-05, 'epoch': 1.98}
{'loss': 0.3628, 'grad_norm': 31.981847763061523, 'learning_rate': 1.8900000000000002e-05, 'epoch': 1.99}
{'loss': 0.0505, 'grad_norm': 8.036585807800293, 'learning_rate': 1.8950000000000003e-05, 'epoch': 1.99}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 0.0, 'grad_norm': 0.0, 'learning_rate': 1.9e-05, 'epoch': 2.0}
{'train_runtime': 338.5773, 'train_samples_per_second': 8.937, 'train_steps_per_second': 1.122, 'train_loss': 0.1421405672976801, 'epoch': 2.0}


Configuration saved in models/nomic_dateaware/final/config.json
Model weights saved in models/nomic_dateaware/final/model.safetensors
tokenizer config file saved in models/nomic_dateaware/final/tokenizer_config.json
Special tokens file saved in models/nomic_dateaware/final/special_tokens_map.json


Model saved successfully.


In [ ]:
!zip -r /content/model.zip /content/models/nomic_dateaware/final

  adding: content/models/nomic_dateaware/final/ (stored 0%)
  adding: content/models/nomic_dateaware/final/model.safetensors (deflated 8%)
  adding: content/models/nomic_dateaware/final/modules.json (deflated 62%)
  adding: content/models/nomic_dateaware/final/1_Pooling/ (stored 0%)
  adding: content/models/nomic_dateaware/final/1_Pooling/config.json (deflated 57%)
  adding: content/models/nomic_dateaware/final/config.json (deflated 60%)
  adding: content/models/nomic_dateaware/final/special_tokens_map.json (deflated 80%)
  adding: content/models/nomic_dateaware/final/tokenizer.json (deflated 71%)
  adding: content/models/nomic_dateaware/final/tokenizer_config.json (deflated 76%)
  adding: content/models/nomic_dateaware/final/vocab.txt (deflated 53%)
  adding: content/models/nomic_dateaware/final/2_Normalize/ (stored 0%)
  adding: content/models/nomic_dateaware/final/sentence_bert_config.json (deflated 4%)
  adding: content/models/nomic_dateaware/final/config_sentence_transformers.json

In [ ]:
from google.colab import files
files.download("/content/model.zip")